# Forecasting Using ARIMA Models in Python

In this notebook, you will learn to use the powerful ARIMA class models to forecast the future. You will learn how to use the statsmodels package to analyze time series, to build tailored models, and to forecast under uncertainty. How will the stock market move in the next 24 hours? How will the levels of CO2 change in the next decade? How many earthquakes will there be next year? You will learn to solve all these problems and more.

## Table of Contents

- [Introduction](#intro)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

path = "data/dc29/"

---
<a id='intro'></a>

<img src="images/ts3_001.png" alt="" style="width: 800px;"/>

<img src="images/ts3_002.png" alt="" style="width: 800px;"/>

<img src="images/ts3_003.png" alt="" style="width: 800px;"/>

<img src="images/ts3_004.png" alt="" style="width: 800px;"/>

<img src="images/ts3_005.png" alt="" style="width: 800px;"/>

<img src="images/ts3_006.png" alt="" style="width: 800px;"/>

<img src="images/ts3_007.png" alt="" style="width: 800px;"/>

<img src="images/ts3_008.png" alt="" style="width: 800px;"/>


## Exploration

In this exercise you will kick off your journey to become an ARIMA master by loading and plotting a time series.

You will be exploring a dataset of monthly US candy production between 1972 and 2018.

Specifically, you are plotting the industrial production index IPG3113N. This is total amount of sugar and confectionery products produced in the USA per month, as a percentage of the January 2012 production. So 120 would be 120% of the January 2012 industrial production.

Check out how this quantity has changed over time and how it changes throughout the year.

In [ ]:
<img src="images/ts3_xxx.png" alt="" style="width: 400px;"/>

In [ ]:
---
<a id='intro'></a>